In [215]:
#http://www.wildml.com/2015/09/implementing-a-neural-network-from-scratch/
import numpy as np
import sklearn
from sklearn import datasets, linear_model
import matplotlib
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler,scale

import pandas as pd
%matplotlib inline  

In [216]:
TRAINING_RATE = 1
TESTING_RATE = 1 - TRAINING_RATE
MISSING_RATE = 0.5
QUERY_RATE = 0.2
# np.random.seed(10)

In [231]:
# Generate a dataset and plot it
np.random.seed(0)
#X : array of shape [n_samples, 2]
# e.g. X[0,:]  is [0.74, 0.46]
# X, y = sklearn.datasets.load_files("../dat/wine_quality") # X is 2 x 1 vector for 200 n_samples , y is 0 or 1
data = pd.read_csv("../dat/wine_quality/winequality-white.csv",sep=';')
# plt.scatter(X[:,0], X[:,1], s=40, c=y, cmap=plt.cm.Spectral)

In [271]:
def preprocessing(d):
    #Add normalization code here if necessary
    min_max_scaler = MinMaxScaler()
    d.ix[:,0:-1] = scale(d.ix[:,0:-1])
#     d.ix[:,0:-1] = min_max_scaler.fit_transform(d.ix[:,0:-1])
    #d['quality'] = d['quality'].apply(lambda x:1.0 if x==6 else 0.0)
    d['quality'] = d['quality'].apply(lambda x: int(x) -3)
    d = d.iloc[np.random.permutation(len(d))]
    t = int(len(d) * TRAINING_RATE)
    tn_data = d.iloc[0:t,:]
    tt_data = d.iloc[t:,:]
    
    tn_X = tn_data.ix[:,0:-1]
    tn_Y = tn_data.ix[:,-1]
    tt_X = tt_data.ix[:,0:-1]
    tt_Y = tt_data.ix[:,-1]
    
    
    return tn_X,tn_Y, tt_X, tt_Y

In [272]:
# tn_X,tn_Y,tt_X,tt_Y = preprocessing(data)
X, y,tt_X,tt_Y = preprocessing(data)
X = X.reset_index(drop=True)
X.head(15)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol
0,-0.064931,-0.577879,0.295920,1.736866,0.239298,1.922443,0.932828,1.779784,1.402345,3.156077,-0.986812
1,-0.301959,-0.875545,0.791757,0.888997,-0.035355,1.040370,0.368039,1.167867,2.197125,2.455025,-0.092863
2,0.409125,-0.379435,-0.365197,-0.668715,-0.447335,-1.076604,-0.502678,0.027628,1.137419,1.666341,-0.336667
3,-1.250071,-1.073988,0.048001,-0.905330,-0.035355,3.568979,0.744565,-1.229644,1.269882,3.594234,1.044891
4,-0.183445,-0.577879,-0.034638,-1.023637,-0.447335,0.040688,-1.232197,-1.152737,-0.584605,2.367393,0.719818
5,0.883181,-1.073988,1.039676,-1.082790,-0.447335,1.628419,0.203309,-0.925358,-1.114458,-0.699710,0.150942
6,0.764667,0.612783,-0.778394,-0.501113,-0.676212,-1.782262,-2.455907,-1.018984,-1.180690,-1.137868,1.044891
7,-0.064931,0.414339,0.791757,-0.944765,0.193523,-1.841067,-1.726388,-0.189720,0.541334,-0.173921,0.150942
8,0.290611,0.315117,-1.356871,-1.063073,-0.172682,-0.841385,0.132710,-0.330160,-0.385910,-0.787342,-0.743008
9,0.646153,-0.081770,-0.613115,1.066458,0.330849,1.157980,0.815164,0.990645,-0.518373,0.965289,-0.580471


In [273]:
# X.ix[0,0]
X = X.as_matrix() # change type "DataFrame" (of Pandas) to numpy array

In [274]:
X

array([[-0.06493106, -0.57787912,  0.29591974, ...,  1.40234528,
         3.15607678, -0.9868119 ],
       [-0.30195909, -0.87554465,  0.79175698, ...,  2.19712525,
         2.45502457, -0.09286267],
       [ 0.40912499, -0.37943543, -0.36519658, ...,  1.13741862,
         1.66634084, -0.33666701],
       ..., 
       [ 0.40912499, -0.28021359,  0.46119882, ..., -1.44561627,
        -0.17392121, -1.23061624],
       [-0.89452915, -0.23060267,  0.87439652, ...,  0.34263866,
         0.43949948, -0.82427568],
       [ 1.59426511, -0.57787912,  0.2132802 , ..., -1.18068962,
        -0.96260494, -0.01159456]])

In [275]:
# Helper function to predict an output (0 or 1)
def predict(model, x):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    # Forward propagation
    z1 = x.dot(W1) + b1
    a1 = sigmoid(z1)
    z2 = a1.dot(W2) + b2 # 200 x 2 -> (200 x 3) * (3 x 2)
    a2 = sigmoid(z2) # 200 x 2
    return np.argmax(a2, axis=1)

In [276]:
def calculate_loss(model):
    W1, b1, W2, b2 = model['W1'], model['b1'], model['W2'], model['b2']
    # Forward propagation
    z1 = X.dot(W1) + b1
    a1 = sigmoid(z1)
    z2 = a1.dot(W2) + b2
    a2 = sigmoid(z2)
    corect_probs = np.square(a2 - X) * (1/2)
    data_loss = np.sum(corect_probs)
    return (1.0/num_examples) * data_loss

In [277]:
# Helper function to plot a decision boundary.
# If you don't fully understand this function don't worry, it just generates the contour plot below.
def plot_decision_boundary(pred_func):
    # Set min and max values and give it some padding
    x_min, x_max = X[:, 0].min() - .5, X[:, 0].max() + .5
    y_min, y_max = X[:, 1].min() - .5, X[:, 1].max() + .5
    h = 0.01
    # Generate a grid of points with distance h between them
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h), np.arange(y_min, y_max, h))
    # Predict the function value for the whole gid
    Z = pred_func(np.c_[xx.ravel(), yy.ravel()])
    Z = Z.reshape(xx.shape)
    # Plot the contour and training examples
    plt.contourf(xx, yy, Z, cmap=plt.cm.Spectral)
    plt.scatter(X[:, 0], X[:, 1], c=y, cmap=plt.cm.Spectral)

In [278]:
num_examples = len(X)
nn_input_dim = 11
nn_output_dim = 11
epsilon = 0.01 #GD update
print(X.shape)

(4898, 11)


In [279]:
def sigmoid(x):                                        
    return 1 / (1 + np.exp(-x))

In [280]:
def build_model(nn_hdim, num_passes=1000, print_loss=False):
    np.random.seed() # X is 200 x 2
    W1 = np.random.randn(nn_input_dim, nn_hdim) / np.sqrt(nn_input_dim) # 2 x 3
    b1 = np.zeros((1, nn_hdim)) # 1 x 3
    W2 = np.random.randn(nn_hdim, nn_output_dim) / np.sqrt(nn_hdim) # 3 x 2
    b2 = np.zeros((1, nn_output_dim)) # 1 x 2
    
    model = {}
    
    # Gradient Descent in batch
    for i in range(0, num_passes):
        
        # Forward propagation
        z1 = X.dot(W1) + b1 #200 x 3 -> (200 x 2) dot (2 x 3)
        a1 = sigmoid(z1) # 200 x 3
        z2 = a1.dot(W2) + b2 # 200 x 2 -> (200 x 3) * (3 x 2)
        a2 = sigmoid(z2) # 200 x 2
        # Backpropagation
        delta2 = (a2 * (1 - a2)) * (a2 - X) # 200 x 2 -> (200 x 2) * (200 x 2) 
        dW2 = (a1.T).dot(delta2) # 3 x 2 -> (3 x 200) dot (200 x 2)
        db2 = np.sum(delta2, axis=0, keepdims=True) # ? x ?
        delta1 = (a1 * (1 - a1)) * (delta2.dot(W2.T)) # 200 x 3 -> (200 x 3) * ((200 x 2) dot (2 x 3))
        dW1 = (X.T).dot(delta1) # 2 x 3 -> (2 x 200) dot (200 x 3)
        db1 = np.sum(delta1, axis=0) # ? x ?
        # Gradient descent parameter update
        W1 += -epsilon * dW1
        b1 += -epsilon * db1
        W2 += -epsilon * dW2
        b2 += -epsilon * db2
        # Assign new parameters to the model
        model = { 'W1': W1, 'b1': b1, 'W2': W2, 'b2': b2}
        if print_loss and i % 100 == 0:
          print( "Loss after iteration %i: %f" %(i, calculate_loss(model)) )
    return model, a2

In [281]:
# Build a model with a 3-dimensional hidden layer
model, a2 = build_model(3, print_loss=True)
 
# Plot the decision boundary -> no use for autoencoder
# plot_decision_boundary(lambda x: predict(model, x))
# plt.title("Decision Boundary for hidden layer size 3")

Loss after iteration 0: 5.497466
Loss after iteration 100: 5.046233
Loss after iteration 200: 5.044744
Loss after iteration 300: 4.863730
Loss after iteration 400: 4.824587
Loss after iteration 500: 4.618550
Loss after iteration 600: 4.612336
Loss after iteration 700: 4.608733
Loss after iteration 800: 4.614546
Loss after iteration 900: 4.612149


In [282]:
a2

array([[  1.02518239e-05,   7.41208343e-07,   4.57917354e-05, ...,
          6.58991547e-07,   1.14225391e-05,   4.86847260e-06],
       [  1.28415797e-05,   1.12246032e-06,   4.89743659e-05, ...,
          1.13438960e-06,   1.76944283e-05,   3.66848270e-05],
       [  2.41881270e-05,   2.69853590e-06,   5.07234044e-06, ...,
          1.84428775e-06,   6.16602917e-06,   3.02437919e-04],
       ..., 
       [  8.23390164e-05,   1.22869037e-05,   1.81768724e-04, ...,
          5.56745751e-06,   2.47804315e-05,   1.50759773e-05],
       [  9.71943089e-06,   6.98911191e-07,   3.59956737e-05, ...,
          6.36540577e-07,   1.05622604e-05,   6.53406302e-06],
       [  6.48902956e-05,   1.29289017e-05,   1.93013095e-05, ...,
          1.05927204e-05,   2.92562272e-05,   1.84367013e-02]])

In [283]:
X

array([[-0.06493106, -0.57787912,  0.29591974, ...,  1.40234528,
         3.15607678, -0.9868119 ],
       [-0.30195909, -0.87554465,  0.79175698, ...,  2.19712525,
         2.45502457, -0.09286267],
       [ 0.40912499, -0.37943543, -0.36519658, ...,  1.13741862,
         1.66634084, -0.33666701],
       ..., 
       [ 0.40912499, -0.28021359,  0.46119882, ..., -1.44561627,
        -0.17392121, -1.23061624],
       [-0.89452915, -0.23060267,  0.87439652, ...,  0.34263866,
         0.43949948, -0.82427568],
       [ 1.59426511, -0.57787912,  0.2132802 , ..., -1.18068962,
        -0.96260494, -0.01159456]])

In [284]:
# np.square(a2 - X) * (1/2)